In [19]:
import pandas as pd
from matplotlib import pyplot as plt

# Importing and cleaning dataset
gamedf = pd.read_csv(r"C:\Users\Zain\Downloads\GameRecommender-main\GameRecommender-main\vgsales.csv")
temp_gamedf = pd.read_csv(r"C:\Users\Zain\Downloads\GameRecommender-main\GameRecommender-main\vgsales.csv")

# Identify the string columns in your dataframe
string_columns = gamedf.select_dtypes(include=['object']).columns

from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder object
label_encoder = LabelEncoder()
print(gamedf.head())
# Perform label encoding on string_columns
for column in string_columns:
    temp_gamedf[column] = label_encoder.fit_transform(temp_gamedf[column])

# Print the modified dataframe
print(temp_gamedf.head())




   Rank                      Name Platform    Year         Genre Publisher  \
0     0                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     1         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     2            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     3         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     4  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  
   Rank   Name  Platform    Year  Genre  Publisher  NA_Sales  EU_Sales  \
0     0  11007        26  2006.0     10        359     41.49     29.02   
1     1   9327   

In [20]:
from sklearn.preprocessing import MinMaxScaler

# Identify the numerical columns in your dataframe
numerical_columns = temp_gamedf.select_dtypes(include=['float64', 'int64', 'int32']).columns
numerical_columns=numerical_columns.drop('Rank')
# Initialize the MinMaxScaler object
scaler = MinMaxScaler()

# Perform normalization on numerical_columns
temp_gamedf[numerical_columns] = scaler.fit_transform(temp_gamedf[numerical_columns])

# Print the modified dataframe with label encoding and normalization
print(temp_gamedf.head())

   Rank      Name  Platform   Year     Genre  Publisher  NA_Sales  EU_Sales  \
0     0  0.957797  0.866667  0.650  0.909091   0.621107  1.000000  1.000000   
1     1  0.811608  0.366667  0.125  0.363636   0.621107  0.700892  0.123363   
2     2  0.484946  0.866667  0.700  0.545455   0.621107  0.382020  0.443832   
3     3  0.957971  0.866667  0.725  0.909091   0.621107  0.379610  0.379394   
4     4  0.639227  0.166667  0.400  0.636364   0.621107  0.271632  0.306340   

   JP_Sales  Other_Sales  Global_Sales  
0  0.368885     0.800378      1.000000  
1  0.666341     0.072848      0.486281  
2  0.370841     0.313150      0.432854  
3  0.320939     0.280038      0.398767  
4  1.000000     0.094607      0.379064  


In [21]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np

# Assuming X contains your features and y contains the target variable (Game Title)
X = temp_gamedf.drop('Name', axis=1).dropna()
y = temp_gamedf['Name'].dropna()
print(X.describe())
# Find the common indices between X and y
common_indices = X.index.intersection(y.index)

# Filter X and y using the common indices
X = X.loc[common_indices]
y = y.loc[common_indices]

# Drop rows containing NaN values
X = X.dropna()

# Initialize the SelectKBest feature selector with f_classif test
selector = SelectKBest(score_func=f_classif, k=5)

# Apply feature selection on the dataset
X_selected = selector.fit_transform(X, y)

# Get the selected feature indices
selected_feature_indices = selector.get_support(indices=True)

# Get the selected feature names
selected_feature_names = X.columns[selected_feature_indices]
print(selected_feature_names)


               Rank      Platform          Year         Genre     Publisher  \
count  16327.000000  16327.000000  16327.000000  16327.000000  16327.000000   
mean    8291.868194      0.526488      0.660161      0.447814      0.506908   
std     4792.669778      0.279071      0.145725      0.341893      0.306924   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%     4135.500000      0.233333      0.575000      0.090909      0.237024   
50%     8294.000000      0.533333      0.675000      0.454545      0.558824   
75%    12440.500000      0.700000      0.750000      0.727273      0.788927   
max    16599.000000      1.000000      1.000000      1.000000      1.000000   

           NA_Sales      EU_Sales      JP_Sales   Other_Sales  Global_Sales  
count  16327.000000  16327.000000  16327.000000  16327.000000  16327.000000  
mean       0.006397      0.005085      0.007697      0.004572      0.006409  
std        0.019802      0.017532      0.030485      0

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import numpy as np

def Similarity(n1, n2, temp_gamedf):
    # Select features for training the models
    features = ['Genre', 'Year', 'Publisher']  # Add other relevant features
    print('Checkpoint 1')
    # Prepare the training data
    X_train = temp_gamedf[features].dropna()
    y_train = temp_gamedf.loc[X_train.index, 'Rank']  # Assuming 'Rank' is your target variable
    print('Checkpoint 2')
    # Initialize the models
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    knn_model = KNeighborsRegressor(n_neighbors=5)
    lr_model = LinearRegression()
    print('Checkpoint 3')
    # Train the models
    rf_model.fit(X_train, y_train)
    knn_model.fit(X_train, y_train)
    lr_model.fit(X_train, y_train)
    print('Checkpoint 4')
    # Prepare the input data for prediction
    input_data = temp_gamedf.loc[[n1, n2], features].dropna()

    print('Checkpoint 5')
    # Handle missing values in the input data
    if input_data.isnull().values.any():
        print('checkpoint 5.5')
        return np.inf  # Return a high similarity for cases with missing values
    print('Checkpoint 6')
    # Predict the target variable (Rank) using each model
    rf_similarity = np.abs(rf_model.predict(input_data.iloc[0].values.reshape(1, -1)) - rf_model.predict(X_train))
    knn_similarity = np.abs(knn_model.predict(input_data.iloc[0].values.reshape(1, -1)) - knn_model.predict(X_train))
    lr_similarity = np.abs(lr_model.predict(input_data.iloc[0].values.reshape(1, -1)) - lr_model.predict(X_train))
    print('Checkpoint 7')
    # Select the highest similarity score among the models
    max_similarity = np.max(np.stack([rf_similarity, knn_similarity, lr_similarity]), axis=0)

    return max_similarity


In [23]:
import numpy as np

def getNeighbors(basegame, K, temp_gamedf):
    # Extract features for the base game
    basegame_features = basegame[['Genre', 'Year', 'Publisher']].values.reshape(1, -1)
    print('Neighbors checkpoint 1')
    # Extract features for all games
    all_game_features = temp_gamedf[['Genre', 'Year', 'Publisher']].values
    print('Neighbors checkpoint 2')
    # Handle missing or non-numeric values in features
    nan_mask = np.isnan(all_game_features.astype(float)).any(axis=1)
    all_game_features = all_game_features[~nan_mask]
    all_game_ranks = temp_gamedf.loc[~nan_mask, 'Rank']
    print('Neighbors checkpoint 3')
    # Calculate similarities using vectorized operations
    similarities = np.abs(Similarity(basegame_features, all_game_features, temp_gamedf))
    print('Neighbors checkpoint 4')
    # Sort and get the indices of the K nearest neighbors
    neighbor_indices = np.argsort(similarities)[:K]
    print('Neighbors checkpoint 5')
    # Extract the neighbor information
    neighbors = [(all_game_ranks.iloc[i], similarities[i]) for i in neighbor_indices]

    return neighbors


In [24]:
def predict_score(temp_gamedf):
    name = 'Call of Duty'
    matching_games = gamedf[gamedf['Name'].str.contains(name)]

    if matching_games.empty:
        print(f"No games found with a name containing '{name}'.")
        return

    new_game = matching_games.iloc[0].to_frame().T
    print('Selected Game:', new_game['Name'].values[0])

    K = 5
    neighbors = getNeighbors(new_game, K, temp_gamedf)
    print('\nRecommended Games:\n')

    for neighbor_info in neighbors:
        neighbor_rank, distance = neighbor_info
        neighbor_game = gamedf[gamedf['Rank'] == neighbor_rank]
        if not neighbor_game.empty:
            print(neighbor_game.iloc[0]['Name'])

# Call the predict_score function with the temp_gamedf DataFrame
predict_score(temp_gamedf)


Selected Game: Call of Duty: Modern Warfare 3
Neighbors checkpoint 1
Neighbors checkpoint 2
Neighbors checkpoint 3
Checkpoint 1
Checkpoint 2
Checkpoint 3
Checkpoint 4


TypeError: unhashable type: 'numpy.ndarray'